# <span style="color:purple; font-weight:bold">K Fold Cross Validation</span>        

A robust method for estimating a model's true performance on unseen data. We can use it to compare perfomances of different ML models on the dataset and choose the best scoring one.               

<span style="color:blue; font-wieght:bold">"Cross-Validation":</span> describes the iterative procedure where:              

    --->  Data roles are swapped in each round.                   
    --->  Every single data point eventually "crosses" from the training set into the validation set (is seen by the model during evaluation) exactly once.


We are familiar with Train/Test Split on the data we feed to our mode.    
But to decide the best splitting ratio we use K Fold Cross validation technique.     
We partition the dataset into k-folds and randomly select parts for training and testing.           


<img src="./assets/images/K_Fold_Splits.webp" width="900" height="500"/>                    

----            



## <span style="color:purple;">Key Concepts</span>            
- <span style="color:blue; font-wieght:bold">k:</span> The number of partitions the data is split into (commonly 5 or 10).            
- <span style="color:blue; font-wieght:bold">Folds:</span> The equal-sized subsets of the dataset.        
- <span style="color:blue; font-wieght:bold">Training Set:</span> The \(k-1\) folds used to fit the model during an iteration.            
- <span style="color:blue; font-wieght:bold">Validation (Test) Set:</span> The single, held-out fold used to evaluate the model's performance in that iteration.             
- <span style="color:blue; font-wieght:bold">Iterative Process:</span> The procedure repeats \(k\) times, ensuring every data point is used for validation exactly once.             
- <span style="color:blue; font-wieght:bold">Averaging:</span> The final performance metric is the average of the scores from all \(k\) iterations.                   
- <span style="color:blue; font-wieght:bold">Stratification (Variation):</span> A technique used to ensure the class distribution within each fold matches the original dataset, vital for imbalanced data.


---       

## <span style="color:purple;">Leave One Out Cross Validation (LOOCV)</span>
In extreme cases model can treat each sample of the dataset as a cell to try split score on.            
Folds(k) = Number of samples (N)                

<img src="./assets/images/Leave_One_Out.avif" width="900" height="500"/>       

---


Video References:                   

https://youtu.be/fSytzGwwBVw?si=izDHb08C1H6ZfY9O                
https://youtu.be/k4vi6Q299Ng?si=FDUHvCIvmgCKQrq1                    
          

---             






In [80]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.datasets import load_digits

digits = load_digits()

In [81]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.3)

**Using different classifiers and checking their perfomance using K-Fold Cross Validation**

**Logistic Regression**

In [82]:
lr = LogisticRegression(max_iter=5000)
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.9703703703703703

**SVM**

In [83]:
svm = SVC(gamma='auto')
svm.fit(X_train, y_train)
svm.score(X_test, y_test)  

0.42962962962962964

**Random Forest**

In [84]:
rf = RandomForestClassifier(n_estimators=40)
rf.fit(X_train, y_train)    
rf.score(X_test, y_test)

0.9629629629629629

## <span style="color:red"> K-Fold Cross Validation

In [85]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)  
kf

KFold(n_splits=3, random_state=None, shuffle=False)

In [86]:
# Taking easy example to show how KFold works
for train_index, test_index in kf.split([1,2,3,4,5,6,7,8.9]):
    print("Train Index:", train_index, "Test Index:", test_index)

Train Index: [3 4 5 6 7] Test Index: [0 1 2]
Train Index: [0 1 2 6 7] Test Index: [3 4 5]
Train Index: [0 1 2 3 4 5] Test Index: [6 7]


In [87]:
# Now using KFold with our digits dataset
# Using generic function to get the score of the passed models and its dataset.
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

---


**Splitting data into X_train, X_test, y_train, y_test using StratifiedKFold** which is a better version of K-fold.          
Standard K-Fold randomly splits the data into 4 equal folds of 4 samples each, without considering the class balance.                   

Check out this blog:
https://neptune.ai/blog/cross-validation-in-machine-learning-how-to-do-it-right             

---

In [88]:
from sklearn.model_selection import StratifiedKFold
skfolds = StratifiedKFold(n_splits=3)       # We will iterate 3 times, each time with different train/test split

scores_logistic = []
scores_svm = []
scores_rf = []

for train_index, test_index in skfolds.split(digits.data, digits.target):
    X_train = digits.data[train_index]
    X_test = digits.data[test_index]    
    y_train = digits.target[train_index]
    y_test = digits.target[test_index]
    
    scores_logistic.append(get_score(lr, X_train, X_test, y_train, y_test))
    scores_svm.append(get_score(svm, X_train, X_test, y_train, y_test))
    scores_rf.append(get_score(rf, X_train, X_test, y_train, y_test))

In [89]:
scores_logistic # 3 scores for 3 iterations

[0.9198664440734557, 0.9415692821368948, 0.9165275459098498]

In [90]:
scores_svm

[0.3806343906510851, 0.41068447412353926, 0.5125208681135225]

In [91]:
scores_rf

[0.9165275459098498, 0.9465776293823038, 0.9298831385642737]

**<span style="color:purple">cross_val_score function**      


Using this sklearn's method to get the score of out three model objects(lr, svm, rf)

In [92]:
from sklearn.model_selection import cross_val_score

In [93]:
cross_val_score(lr, digits.data, digits.target, cv=3)  # 3 iterations

array([0.91986644, 0.94156928, 0.91652755])

In [94]:
cross_val_score(svm, digits.data, digits.target, cv=3) # 3 iterations

array([0.38063439, 0.41068447, 0.51252087])

In [95]:
cross_val_score(rf, digits.data, digits.target, cv=3)  # 3 iterations

array([0.93823038, 0.94323873, 0.91986644])

**Tuning Parameter using K Fold Cross Validation**

In [96]:
# Lets tune 'cv' paramter for RandomForest model until we get a good score
scores_1 = cross_val_score(rf, digits.data, digits.target, cv=3)
np.average(scores_1)

np.float64(0.9376739009460211)

In [97]:
scores_2 = cross_val_score(rf, digits.data, digits.target, cv=3)
np.average(scores_2)

np.float64(0.9360044518642181)

In [98]:
scores_3 = cross_val_score(rf, digits.data, digits.target, cv=3)
np.average(scores_3)  # Best result so far so we can consider cv=3 for our model

np.float64(0.9309961046188091)